Assuming we have a finite number of reads, how will the organisms at different abundances be represented?  
For example, assume 3 organisms at the following abundances based on cell copy number:

||abundance (%)|
|:-|:-|
|a|50|
|b|25|
|c|25|

Working backwards from flow-cell binding, how many reads will belong to these three organism? Assume 1000 reads total.

In [2]:
1000*0.5

500.0